In [72]:
import numpy as np
from scipy import sparse 
from sklearn.metrics import precision_score, recall_score, f1_score


In [73]:
N = 15000 # Rows
d = 16 # Features
C = 26 # Class numbers

In [74]:
def read_data(file_path):
    X, Y = [], []
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split(' ')
            label = int(data[0])  # Nhãn là số đầu tiên trong mỗi dòng
            features = [float(feature.split(':')[1]) for feature in data[1:]]  # Các đặc trưng từ 1 đến 16
            X.append(features)
            Y.append(label)
    return np.array(X), np.array(Y)

In [75]:
file_train_path = "./letter.scale.tr"
file_test_path = "./letter.scale.t"
file_val_path = "./letter.scale.val"

# Training dataset
X_train, y_train = read_data(file_train_path)
X_train = X_train.T
print(X_train.shape)
print(y_train.shape)

# Testing dataset
X_test, y_test = read_data(file_test_path)
X_test = X_test.T
print(X_test.shape)
print(y_test.shape)

# Validation dataset
X_val, y_val= read_data(file_val_path)
X_val = X_val.T
print(X_val.shape)
print(y_val.shape)

(16, 10500)
(10500,)
(16, 5000)
(5000,)
(16, 4500)
(4500,)


In [76]:
# Trừ đi 1 bởi vì trong dữ liệu label bắt đầu từ 1 -> 26: 
# Sau khi trừ 1 thì đảm bảo label từ 0 -> 25
y_train = y_train - 1
y_test = y_test - 1
y_val = y_val - 1 

In [77]:
def convert_labels(y, C=C):
    """
    Chuyển đổi mảng nhãn thành dạng ma trận one-hot encoding.

    Parameters:
        y (array): Mảng chứa các nhãn.
        C (int): Số lớp (số cột) trong ma trận one-hot encoding. Mặc định là số lớp C được định nghĩa trước đó.

    Returns:
        array: Ma trận one-hot encoding của các nhãn.
    """
    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape=(C, len(y))).toarray()
    return Y 


def loss(X, Y_hat, theta):
    Y = softmax_stable(np.dot(theta.T, X))
    eps = 1e-15
    Y_hat = np.clip(Y_hat, eps, 1 - eps)

    # Tính toán cross entropy loss
    loss = -np.sum(Y * np.log(Y_hat)) / Y.shape[1]
    return loss

def softmax(Z):
    """
    Tính hàm softmax cho ma trận đầu vào Z.

    Parameters:
        Z (array): Ma trận đầu vào.

    Returns:
        array: Ma trận chứa kết quả của hàm softmax.
    """
    e_Z = np.exp(Z)
    A = e_Z / e_Z.sum(axis=0)
    return A

def softmax_stable(Z):
    """
    Tính hàm softmax ổn định cho ma trận đầu vào Z.

    Parameters:
        Z (array): Ma trận đầu vào.

    Returns:
        array: Ma trận chứa kết quả của hàm softmax ổn định.
    """
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    A = e_Z / e_Z.sum(axis=0)
    return A


In [78]:
def accuracy_score(y_true, y_pred):
    """
    Tính độ chính xác của dự đoán.
    
    Parameters:
        y_true (array): Nhãn thực sự.
        y_pred (array): Nhãn dự đoán.
    
    Returns:
        float: Độ chính xác của dự đoán, được tính dưới dạng phần trăm.
    """
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy

def predict(theta, X):
    """
    Dự đoán nhãn của các mẫu đầu vào bằng mô hình softmax.
    
    Parameters:
        theta (array): Ma trận trọng số.
        X (array): Ma trận các mẫu đầu vào.
    
    Returns:
        array: Nhãn dự đoán cho các mẫu đầu vào.
    """
    A = softmax_stable(np.dot(theta.T, X))
    return np.argmax(A, axis=0)


def calculate_f1_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1



#### Hàm với cỡ bước hằng

In [79]:
def softmax_regression(X, y, theta_init, eta, tol=1e-8, max_count=500000):
    theta = theta_init
    C = theta.shape[1]
    Y = convert_labels(y, C)
    N = X.shape[1]
    d = X.shape[0]
    
    count = 0
    prev_theta = theta

    while count < max_count:
        # Shuffle data
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(theta.T, xi))
            grad = xi.dot((ai - yi).T)
            theta_new = theta - eta * grad
            count += 1
            
            # Check for convergence
            if np.linalg.norm(theta_new - prev_theta) < tol:
                return theta_new
            
            prev_theta = theta
            theta = theta_new
    
    return theta

#### Backtracking

In [ ]:
def backtraking_softmax(X_train, y_train, eta=1, m=0.5, alpha=0.5, batch_size=1024, epochs=256):
    y_train1hot = covert_labels(y_train, 26)
    d, C = X_train.shape[0], y_train1hot.shape[0]
    theta_init = np.random.randn(d, C)
    theta = [theta_init]

    for epoch in range(epochs):
        for i in range(0, X_train.shape[1], batch_size):
            # Get the batch
            X_batch = X_train[:, i:i+batch_size]
            y_batch = y_train1hot[:, i:i+batch_size]

            # Compute the gradient over the batch
            ai = softmax_stable(np.dot(theta[-1].T, X_batch))
            grad = np.dot(X_batch, (ai - y_batch).T) / batch_size

            # Update theta with backtracking line search
            theta_new = theta[-1] - eta * grad
            t_k = eta
            j = 0
            while (loss(X_train, y_train1hot, theta_new) > loss(X_train, y_train1hot, theta[-1]) - m * t_k * np.linalg.norm(grad) ** 2) and (j < 5):
                t_k *= alpha
                theta_new = theta[-1] - t_k * grad
                j += 1

            # Append the new theta to the list
            theta.append(theta_new)
            if np.linalg.norm(theta[-1] - theta[-2]) < 1e-8:
                print(f"Converged at epoch {epoch + 1}, iteration {i // batch_size + 1}")
                return theta[-1]

        print(f"Epoch {epoch + 1}, Loss: {loss(X_train, y_train1hot, theta[-1])}")

    # Final trained theta
    return theta[-1]

# Khởi tạo mô hình và thực hiện huấn luyện mô hình

## Cỡ bước hằng

In [80]:
eta = 0.03
theta_init = np.random.randn(d, C)
theta = softmax_regression(X_train, y_train, theta_init, eta)

In [81]:
print(theta)

[[ 1.88501781e+00 -3.88669282e+00  1.95927367e+00 -6.70307773e+00
  -3.46994370e+00 -2.19348469e+00  5.59219282e-01 -3.72411275e+00
   4.06262205e+00  8.00802768e-01  3.07447379e-01  8.94228769e-01
   2.01929006e+00 -8.84638022e-01 -1.79335821e+00  1.55769810e+00
   3.36229571e+00 -1.66386626e+00 -7.30402301e-03  2.75639627e+00
   4.27822064e-01  2.08203623e+00  4.42576480e+00 -4.22983615e-01
   4.13621221e+00 -9.58070985e-01]
 [-2.98590036e+00  2.36207593e+00  1.32532341e+00  4.97967988e+00
   1.19852941e+00 -9.56621221e-01  5.68393998e-01  8.79163368e-01
  -7.05822995e-01 -3.59418940e+00 -1.36923288e+00  4.72436712e-01
  -6.18988838e-01 -2.09659537e-01  1.80075249e+00 -3.25452999e+00
  -2.59780429e+00  7.87164870e-01 -8.98249251e-01  1.26065400e+00
   2.16140259e+00  5.07993595e-01 -2.30362522e+00 -1.34374182e+00
  -2.65773813e+00 -4.24161220e+00]
 [ 3.08431212e+00 -1.54204028e+00 -2.19266037e+00 -1.58592464e+00
  -2.93003322e+00  3.13419090e-01  2.54440724e+00  9.91270381e+00
  -1.0

### Dự đoán mô hình với cỡ bước hằng

In [28]:
# Dự đoán trên tập huấn luyện
Y_train_pred = predict(theta, X_train)  
print("Các lớp được dự đoán cho tập huấn luyện với 20 dữ liệu đầu:\n", Y_train_pred[:20])
print("Các lớp thực sự cho tập huấn luyện với 20 dữ liệu đầu:\n", y_train[:20])
acc_train = accuracy_score(y_train.T, Y_train_pred)
print(f"Độ chính xác cho tập huấn luyện: {acc_train:.2f}%")

# Tính Precision, Recall và F1 score trên tập huấn luyện
precision, recall, f1 = calculate_f1_score(y_train, Y_train_pred)

print(f'Precision có trọng số trên tập huấn luyện: {precision:.2f}')
print(f'Recall có trọng số trên tập huấn luyện: {recall:.2f}')
print(f'F1 score có trọng số trên tập huấn luyện: {f1:.2f}')


Các lớp được dự đoán cho tập huấn luyện với 20 dữ liệu đầu:
 [21 11  8  2 21 10  9  3  7  6  9  4  3  2 21 18 24 11 23 19]
Các lớp thực sự cho tập huấn luyện với 20 dữ liệu đầu:
 [21 11  8  2 21 10  9  3  7  6  9  2  3  6 21  0 24 11 23 19]
Độ chính xác cho tập huấn luyện: 75.61%
Precision có trọng số trên tập huấn luyện: 0.75
Recall có trọng số trên tập huấn luyện: 0.76
F1 score có trọng số trên tập huấn luyện: 0.75


In [29]:
# Dự đoán trên tập kiểm thử
Y_test_pred = predict(theta, X_test)
print("Các lớp được dự đoán cho tập kiểm thử cho 20 dữ liệu đầu:\n", Y_test_pred[:20])
print("Các lớp thực sự cho tập kiểm thử cho 20 dữ liệu đầu:\n", y_test[:20])

acc_test = accuracy_score(y_test.T, Y_test_pred)
print(f"Độ chính xác cho tập kiểm thử: {acc_test:.2f}%")


# Tính Precision, Recall và F1 score trên tập kiểm thử
test_precision, test_recall, test_f1 = calculate_f1_score(y_test, Y_test_pred)

print(f'Precision có trọng số trên tập kiểm thử: {test_precision:.2f}')
print(f'Recall có trọng số trên tập kiểm thử: {test_recall:.2f}')
print(f'F1 có trọng số trên tập kiểm thử: {test_f1:.2f}')

Các lớp được dự đoán cho tập kiểm thử cho 20 dữ liệu đầu:
 [16 17  5  5 19 19 16  9  2 14 21  8 13 17  4 16  5 23  5  4]
Các lớp thực sự cho tập kiểm thử cho 20 dữ liệu đầu:
 [16 17  5  5 19 19  6 25  2 14 21  8  7  3  4 16 15 23  5  4]
Độ chính xác cho tập kiểm thử: 75.08%
Precision có trọng số trên tập kiểm thử: 0.75
Recall có trọng số trên tập kiểm thử: 0.75
F1 có trọng số trên tập kiểm thử: 0.75


In [30]:
# Dự đoán trên tập thẩm định
Y_val_pred = predict(theta, X_val)
print("Các lớp được dự đoán cho tập thẩm định cho 20 dữ liệu đầu:\n", Y_val_pred[:20])
print("Các lớp thực sự cho tập thẩm định cho 20 dữ liệu đầu:\n", y_val[:20])

acc_test = accuracy_score(y_val, Y_val_pred.T)
print(f"Độ chính xác cho tập thẩm định: {acc_test:.2f}%")

# Tính Precision, Recall và F1 score trên tập thẩm định
val_precision, val_recall, val_f1 = calculate_f1_score(y_val, Y_val_pred)

print(f'Precision có trọng số trên tập thẩm định: {val_precision:.2f}')
print(f'Recall có trọng số trên tập thẩm định: {val_recall:.2f}')
print(f'F1 score có trọng số trên tập thẩm định: {val_f1:.2f}')

Các lớp được dự đoán cho tập thẩm định cho 20 dữ liệu đầu:
 [13  5  2 10  9 11  0  1 17  4 25  1 15 11 13 17  5 12  4 19]
Các lớp thực sự cho tập thẩm định cho 20 dữ liệu đầu:
 [13  5  6 17  9 11  0  1 18 25 18  1 15 11 13 17  5 12  4 19]
Độ chính xác cho tập thẩm định: 75.22%
Precision có trọng số trên tập thẩm định: 0.75
Recall có trọng số trên tập thẩm định: 0.75
F1 score có trọng số trên tập thẩm định: 0.75


## Backtracking (Armijo step size)

In [59]:
backtracking_theta = backtraking_softmax(X_train, y_train, eta=1, m=0.5, alpha=0.5, batch_size=1024, epochs=256)

Epoch 1, Loss: 33.16045732665291
Epoch 2, Loss: 33.144123032225366
Epoch 3, Loss: 33.12822025718436
Epoch 4, Loss: 33.11267615627507
Epoch 5, Loss: 33.09743436884965
Epoch 6, Loss: 33.082449184699335
Epoch 7, Loss: 33.06768182279427
Epoch 8, Loss: 33.053098200097075
Epoch 9, Loss: 33.03866764896077
Epoch 10, Loss: 33.02436219262147
Epoch 11, Loss: 33.01015612717938
Epoch 12, Loss: 32.99602576143215
Epoch 13, Loss: 32.98194923352935
Epoch 14, Loss: 32.96790636397054
Epoch 15, Loss: 32.92774640301987
Epoch 16, Loss: 32.887892276038535
Epoch 17, Loss: 32.80198203646321
Epoch 18, Loss: 32.601982446755706
Epoch 19, Loss: 32.3031404406124
Epoch 20, Loss: 31.965991772614924
Epoch 21, Loss: 31.619066700777346
Epoch 22, Loss: 31.276407507500423
Epoch 23, Loss: 30.94208583139057
Epoch 24, Loss: 30.616846543893327
Epoch 25, Loss: 30.300852256946992
Epoch 26, Loss: 29.994417833632713
Epoch 27, Loss: 29.698042233897844
Epoch 28, Loss: 29.412221368988334
Epoch 29, Loss: 29.13725439922613
Epoch 30, L

### Dự đoán mô hình với thuật toán quay lui

In [67]:
# Dự đoán trên tập huấn luyện với thuật toán quay lui
Y_train_pred = predict(backtracking_theta, X_train)
print("Các lớp được dự đoán cho tập huấn luyện với 20 dữ liệu đầu với thuật toán quay lui:\n", Y_train_pred[:30])
print("Các lớp thực sự cho tập huấn luyện với 20 dữ liệu đầu với thuật toán quay lui:\n", y_train[:30])

acc_train = accuracy_score(y_train.T, Y_train_pred)
print(f"Độ chính xác cho tập huấn luyện với thuật toán quay lui: {acc_train:.2f}%")

# Tính Precision, Recall và F1 score trên tập huấn luyện với thuật toán quay lui
precision, recall, f1 = calculate_f1_score(y_train, Y_train_pred)

print(f'Precision có trọng số trên tập huấn luyện với thuật toán quay lui: {precision:.2f}')
print(f'Recall có trọng số trên tập huấn luyện với thuật toán quay lui: {recall:.2f}')
print(f'F1 score có trọng số trên tập huấn luyện với thuật toán quay lui: {f1:.2f}')


Các lớp được dự đoán cho tập huấn luyện với 20 dữ liệu đầu với thuật toán quay lui:
 [21 11  8  2 21 16  9  3  7 10 14  4  3  2 21 18 24 11 23 19 24 16 12 16
 20 21 13 21 17 13]
Các lớp thực sự cho tập huấn luyện với 20 dữ liệu đầu với thuật toán quay lui:
 [21 11  8  2 21 10  9  3  7  6  9  2  3  6 21  0 24 11 23 19 24  6 12 16
 20 21 20 24 23 13]
Độ chính xác cho tập huấn luyện với thuật toán quay lui: 72.12%
Precision có trọng số trên tập huấn luyện với thuật toán quay lui: 0.72
Recall có trọng số trên tập huấn luyện với thuật toán quay lui: 0.72
F1 score có trọng số trên tập huấn luyện với thuật toán quay lui: 0.72


In [68]:
# Dự đoán trên tập kiểm thử với thuật toán quay lui
Y_test_pred = predict(backtracking_theta, X_test)
print("Các lớp được dự đoán cho tập kiểm thử với 20 dữ liệu đầu với thuật toán quay lui:\n", Y_test_pred[:20])
print("Các lớp thực sự cho tập kiểm thử với 20 dữ liệu đầu với thuật toán quay lui:\n", y_train[:20])
acc_test = accuracy_score(y_test.T, Y_test_pred)
print(f"Độ chính xác cho tập kiểm thử với thuật toán quay lui: {acc_test:.2f}%")

# Tính Precision, Recall và F1 score trên tập kiểm thử với thuật toán quay lui
test_precision, test_recall, test_f1 = calculate_f1_score(y_test, Y_test_pred)

print(f'Precision có trọng số trên tập kiểm thử với thuật toán quay lui: {test_precision:.2f}')
print(f'Recall có trọng số trên tập kiểm thử với thuật toán quay lui: {test_recall:.2f}')
print(f'F1 có trọng số trên tập kiểm thử với thuật toán quay lui: {test_f1:.2f}')


Các lớp được dự đoán cho tập kiểm thử với 20 dữ liệu đầu với thuật toán quay lui:
 [14 17  5  5 19 19  1  5  2 14 21  8  7 17 11  1  5 23  5  4]
Các lớp thực sự cho tập kiểm thử với 20 dữ liệu đầu với thuật toán quay lui:
 [21 11  8  2 21 10  9  3  7  6  9  2  3  6 21  0 24 11 23 19]
Độ chính xác cho tập kiểm thử với thuật toán quay lui: 71.92%
Precision có trọng số trên tập kiểm thử với thuật toán quay lui: 0.72
Recall có trọng số trên tập kiểm thử với thuật toán quay lui: 0.72
F1 có trọng số trên tập kiểm thử với thuật toán quay lui: 0.71


In [69]:
# Dự đoán trên tập thẩm định với thuật toán quay lui
Y_val_pred = predict(backtracking_theta, X_val)
print("Các lớp được dự đoán cho tập thẩm định với 20 dữ liệu đầu với thuật toán quay lui:\n", Y_val_pred[:20])
print("Các lớp thực sự cho tập thẩm định với 20 dữ liệu đầu với thuật toán quay lui:\n", y_val[:20])

acc_val = accuracy_score(y_val.T, Y_val_pred)
print(f"Độ chính xác cho tập thẩm định với thuật toán quay lui: {acc_val:.2f}%")

# Tính Precision, Recall và F1 score trên tập thẩm định với thuật toán quay lui
val_precision, val_recall, val_f1 = calculate_f1_score(y_val, Y_val_pred)

print(f'Precision có trọng số trên tập thẩm định với thuật toán quay lui: {val_precision:.2f}')
print(f'Recall có trọng số trên tập thẩm định với thuật toán quay lui: {val_recall:.2f}')
print(f'F1 score có trọng số trên tập thẩm định với thuật toán quay lui: {val_f1:.2f}')

Các lớp được dự đoán cho tập thẩm định với 20 dữ liệu đầu với thuật toán quay lui:
 [13  5  2 17  9 11  0  1 17 25 18  1 15 11 12 17  5 12  4 19]
Các lớp thực sự cho tập thẩm định với 20 dữ liệu đầu với thuật toán quay lui:
 [13  5  6 17  9 11  0  1 18 25 18  1 15 11 13 17  5 12  4 19]
Độ chính xác cho tập thẩm định với thuật toán quay lui: 71.87%
Precision có trọng số trên tập thẩm định với thuật toán quay lui: 0.72
Recall có trọng số trên tập thẩm định với thuật toán quay lui: 0.72
F1 score có trọng số trên tập thẩm định với thuật toán quay lui: 0.71
